# Day17: BERT在GLUE MRPC任务上的微调
本项目演示如何使用`bert-base-uncased`模型在GLUE的MRPC（Microsoft Research Paraphrase Corpus）任务上进行微调。
- 任务目标：判断两个句子是否为同义句（paraphrase）。
- 评估指标：准确率、F1分数。

**流程：**
1. 数据加载与探索
2. 数据预处理与分词
3. DataLoader构建
4. 模型定义
5. 训练与评估


见scripts/DL/day17.py。该脚本无法在CPU上训练推理。


现在把SimpleLSTM做的情绪推理模型push到hub。


In [4]:
from huggingface_hub import notebook_login

notebook_login()

最直接的方式是直接在TrainingArgument中添加push_to_hub=True，如    
```python
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-mrpc", save_strategy="epoch", push_to_hub=True
)

trainer.push_to_hub()
```

## 使用huggingface_hub


现在cli输入huggingface-cli login并输入access token

### upload_file上传小文件

In [ ]:
from huggingface_hub import create_repo,upload_file

In [8]:
create_repo("lstm-emotion-detection", exist_ok=True)

RepoUrl('https://huggingface.co/Dove667/lstm-emotion-detection', endpoint='https://huggingface.co', repo_type='model', repo_id='Dove667/lstm-emotion-detection')

In [12]:
upload_file(
    path_or_fileobj="/home/dove/DLModel/SimpleLSTM/checkpoint_9.pth",
    path_in_repo="model.pth",
    repo_id="dove667/lstm-emotion-detection",
)

checkpoint_9.pth:   0%|          | 0.00/48.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Dove667/lstm-emotion-detection/commit/cd2ff86aac8b8b58a67252c04075f298b5c0392e', commit_message='Upload model.pth with huggingface_hub', commit_description='', oid='cd2ff86aac8b8b58a67252c04075f298b5c0392e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dove667/lstm-emotion-detection', endpoint='https://huggingface.co', repo_type='model', repo_id='Dove667/lstm-emotion-detection'), pr_revision=None, pr_num=None)

### Relository 类
当文件大小大于5GB时，不能使用upload_file方法，可以使用Repository类（预先安装git和git-lfs）
```bash
sudo apt-get update
sudo apt-get install git-lfs
git lfs install
```

In [ ]:
from huggingface_hub import Repository

repo = Repository("<path_to_dummy_folder>", clone_from="<namespace>/dummy-model")

然后可以调用一系列git方法：
```python
repo.git_pull()

model.save_pretrained("<path_to_dummy_folder>")
tokenizer.save_pretrained("<path_to_dummy_folder>")

repo.git_add()
repo.git_commit("Add model and tokenizer files")
repo.git_push()

### 传统git的方法
```bash
git clone https://huggingface.co/<namespace>/<your-model-id>
```

对模型任意操作，比如微调：
```python
from transformers import AutoModelForMaskedLM, AutoTokenizer

checkpoint = "camembert-base"

model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Do whatever with the model, train it, fine-tune it...

model.save_pretrained("<path_to_dummy_folder>")
tokenizer.save_pretrained("<path_to_dummy_folder>")
```

然后正常使用git 
```bash
git add . 
git commit -m ""
git push
```